# Capítulo 5 - Operações estruturadas - Parte 1
<br>
<div style="text-align: justify">Por definição, um DataFrame consiste em uma série de registros (como linhas em uma tabela), que são do tipo Linha e um número de colunas (como colunas em uma planilha) que representam uma expressão de cálculo que pode ser executada em cada registro individual do conjunto de dados. Os esquemas definem o nome e o tipo de dados em cada coluna. O particionamento do DataFrame define o layout da distribuição física do DataFrame ou do Conjunto de Dados no cluster. O esquema de particionamento define como isso é alocado. Você pode definir isso para ser baseado em valores em uma determinada coluna ou não-deterministicamente.</div>

### Sumário
 1. __.schema__ 
 2. __Colunas e expressões__
 3. __col() / column()__ - Colunas
 4. __expr()__ Expressões 
 5. __Colunas como expressões__
 6. __first()__ Registros e linhas 
 7. __Row()__ - Criando Linhas 
 8. __createDataFrame()__ - Criando DataFrames 
 9. __select() / selectExpr()__ -  
 10. __lit()__ - Convertendo para tipos Spark (Literais) 
 11. __withColumn()__ - Adicionando colunas
 12. __withColumnRenamed()__ - Renomeando colunas
 13. __Caracteres reservados e palavras chaves__

In [2]:
# Abrindo um Dataframe de trabalho
path = "file:///root/sparkcurso/Spark_Definitive_Guide/data/flight-data/json/2015-summary.json"
df = spark.read.format("json").load(path)

In [3]:
# Mostrando as 5 primeiras linhas do dataframe
df.show(5)
# Mostrando as 5 primeiras linhas do dataframe mostrando o conteúdo completo da célula
df.show(5, truncate=False )

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



### Schema
<br>
<div style="text-align: justify">Um schema define os nomes e tipos de coluna de um DataFrame. Podemos deixar uma fonte de dados definir o schema (chamado schema-on-read) ou podemos defini-lo explicitamente por nós mesmos.

Um esquema é um _StructType_ composto de vários campos, _StructFields_, que possuem um nome, tipo, um sinalizador Booleano que especifica se essa coluna pode conter valores ausentes ou nulos e, finalmente, os usuários podem especificar opcionalmente metadados associados a essa coluna. Os metadados são uma maneira de armazenar informações sobre essa coluna (o Spark usa isso em sua biblioteca de aprendizado de máquina).</div>

In [4]:
# Mostrando o Schema do DataFrame
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [5]:
# Descrevendo a estrutura do DataFrame
df.describe()

DataFrame[summary: string, DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: string]

### Colunas e expressões
<br>
<div style="text-align: justify">As colunas no Spark são semelhantes às colunas de uma planilha, do dataframe R ou do DataFrame do pandas. Você pode selecionar, manipular e remover colunas de DataFrames e essas operações são representadas como expressões. Para Spark, as colunas são construções lógicas que simplesmente representam um valor calculado por registro por meio de uma expressão. Isso significa que para ter um valor real para uma coluna, precisamos ter uma linha; e para ter uma linha, precisamos ter um DataFrame. Você não pode manipular uma coluna individual fora do contexto de um DataFrame; você deve usar as transformações do Spark em um DataFrame para modificar o conteúdo de uma coluna.</div>

In [20]:
# Há várias maneiras diferentes de construir e referenciar colunas, mas as duas maneiras mais simples 
# são usar as funções col ou column. 
# Para usar uma dessas funções, você passa um nome de coluna:
from pyspark.sql.functions import col, column
col("someColumnName")
column("someColumnName")

Column<someColumnName>

### Referências explícitas a colunas
<br>
<div style="text-align: justify">Se você precisar se referir a uma coluna específica do DataFrame, poderá usar o método col no DataFrame específico. Isso pode ser útil quando você está executando uma junção e precisa se referir a uma coluna específica em um DataFrame que pode compartilhar um nome com outra coluna no DataFrame associado. Como um benefício adicional, o Spark não precisa resolver essa coluna porque fizemos isso para o Spark:</div>

In [ ]:
df.col("count")

### Expressões
<br>
<div style="text-align: justify">Nós mencionamos anteriormente que colunas são expressões, mas o que é uma expressão? Uma expressão é um conjunto de transformações em um ou mais valores em um registro em um DataFrame. Pense nisso como uma função que usa como entrada um ou mais nomes de coluna, resolve-os e aplica-se potencialmente mais expressões para criar um único valor para cada registro no conjunto de dados. É importante ressaltar que esse “valor único” pode na verdade ser um tipo complexo, como um Map ou Array.<div>

#### Colunas como expressões
<br>
<div style="text-align: justify">Colunas fornecem um subconjunto da funcionalidade de expressão. Se você usar col () e desejar executar transformações nessa coluna, deverá executar aquelas na referência dessa coluna. Ao usar uma expressão, a função <b>expr</b> pode realmente analisar transformações e referências de coluna de uma seqüência de caracteres e, posteriormente, pode ser passada para outras transformações. Vamos ver alguns exemplos:</div>

expr ("someCol - 5") é a mesma transformação que executar col ("someCol") - 5, ou mesmo<br>
expr ("someCol") - 5. Isso ocorre porque o Spark compila esses itens em uma árvore lógica especificando a ordem das operações. Isso pode ser um pouco confuso no começo, mas lembre-se de alguns pontos importantes:
- Colunas são apenas expressões.
- Colunas e transformações dessas colunas são compiladas para o mesmo plano lógico que as expressões analisadas.

In [ ]:
(((col("someCol") + 5) * 200) - 6) < col("otherCol")

In [ ]:
expr("(((someCol + 5) * 200) - 6) < otherCol")

### Acessando as colunas de um dataFrame
Às vezes, você precisa ver as colunas do DataFrame, o que pode ser feito usando algo como printSchema; no entanto, se você quiser acessar programaticamente colunas, poderá usar a propriedade columns para ver todas as colunas em um DataFrame:

In [6]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

### Registros e linhas
No Spark, cada linha em um DataFrame é um único registro. Spark representa esse registro como um objeto do tipo Row. Spark manipula objetos Row usando expressões de coluna para produzir valores utilizáveis. Objetos de linha representam internamente matrizes de bytes. A interface de matriz de bytes nunca é mostrada aos usuários porque usamos apenas expressões de coluna para manipulá-las.

In [7]:
# Lendo o primeiro registro(linha) do DataFrame
df.first()

Row(DEST_COUNTRY_NAME=u'United States', ORIGIN_COUNTRY_NAME=u'Romania', count=15)

### Criando Linhas

Você pode criar linhas instanciando manualmente um objeto _Row_ com os valores que pertencem a cada coluna. É importante notar que apenas os DataFrames têm esquemas. As próprias linhas não possuem esquemas. Isso significa que, se você criar uma Linha manualmente, deverá especificar os valores na mesma ordem que o esquema do DataFrame ao qual eles podem ser anexados. 

In [14]:
# Criando um registro
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

print myRow[0]
print myRow[1]
print myRow[2]
print myRow[3]

Hello
None
1
False


### Criando DataFrames

Também podemos criar DataFrames na hora, pegando um conjunto de linhas e convertendo-as em um DataFrame.

In [17]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

# Criando o DataFrame
myManualSchema = StructType([
StructField("some", StringType(), True),
StructField("col", StringType(), True),
StructField("names", LongType(), False)
])

# Criando a linha(Registro)
myRow = Row("Hello", None, 1)

# Inserindo o resgistro no DataFrame
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



### select / selectExpr

__select__ e __selectExpr__ permitem que você faça consultas SQL em DataFrames equivalentes a uma tabela de dados.

In [18]:
# Selecionando uma coluna no DataFrame e exibindo os dois primeiros campos
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [19]:
# Selecionando duas colunas no DataFrame e exibindo os dois primeiros campos
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [20]:
# Apresentando diversas formas de referenciar uma coluna
from pyspark.sql.functions import expr, col, column
df.select(
expr("DEST_COUNTRY_NAME"),
col("DEST_COUNTRY_NAME"),
column("DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



Como vimos até agora, a __expr__ é a referência mais flexível que podemos usar. Pode se referir a uma coluna simples ou a uma manipulação de string de uma coluna. Para ilustrar, vamos alterar o nome da coluna e, em seguida, alterá-lo novamente usando a palavra-chave __AS__ e, em seguida, o método de __alias__ na coluna.

In [21]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)
# Equivalente em SQL: 
# SELECT DEST_COUNTRY_NAME as destination FROM dfTable LIMIT 2 

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [22]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME")).show(2)
# Retorna o nome da coluna alterado por expr() ao valor original

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



Como o __select__ seguido por uma série de __expr__ é um padrão tão comum, o Spark tem uma forma abreviada de fazer isso de maneira eficiente: __selectExpr__.

In [23]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)
# Retorna o nome da coluna alterado por expr() ao valor original

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



Podemos tratar o __selectExpr__ como uma maneira simples de criar expressões complexas que criam novos DataFrames. Na verdade, podemos adicionar qualquer instrução SQL não agregada válida e, contanto que as colunas sejam resolvidas, ela será válida

In [24]:
df.selectExpr(
"*", # todas as colunas originais
"(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [25]:
# Com a expressão select, também podemos especificar agregações em todo o DataFrame aproveitando as funções que temos.
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



### Convertendo para tipos Spark (Literais)

Às vezes, precisamos passar valores explícitos para o Spark, que são apenas um valor (em vez de uma nova coluna). Isso pode ser um valor constante ou algo que precisaremos comparar para mais tarde. A maneira como fazemos isso é através de literais. Isso é basicamente uma tradução do valor literal de uma dada linguagem de programação para uma que o Spark entenda.

In [26]:
# Selecionando todas as colunas do DataFrame e adicionando uma coluna chamada "One" com o valor literal 1.
from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)

# Equivalente em SQL:
# SELECT *, 1 as One FROM dfTable LIMIT 2

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



### withColumn (Adicionando colunas)

Há também uma maneira mais formal de adicionar uma nova coluna a um DataFrame, usando o método withColumn em nosso DataFrame.

In [27]:
# Adicionando nova coluna "nunberOne" com o valor literal 1 em um DataFrame
df.withColumn("numberOne", lit(1)).show(2)

# Equivalente em SQL:
# SELECT *, 1 as numberOne FROM dfTable LIMIT 2

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [28]:
# Inserindo uma coluna com uma regra que seleciona elementos onde o país de origem é igual ao país de destino
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [29]:
# Observe que a função withColumn aceita dois argumentos: o nome da coluna e a expressão 
# que criará o valor para essa linha no DataFrame. 
# Curiosamente, também podemos renomear uma coluna dessa maneira.

df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

### withColumnRenamed (Renomeando colunas)

Há também uma maneira mais formal de adicionar uma nova coluna a um DataFrame, usando o método withColumn em nosso DataFrame.

In [30]:
# Renomeando a coluna "DEST_COUNTRY_NAME" para "dest"
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

### Caracteres reservados e palavras chaves

Uma coisa que você pode encontrar é caracteres reservados, como espaços ou traços em nomes de colunas. Manipular estes significa escapar nomes de coluna apropriadamente. No Spark, fazemos isso usando caracteres (`). Vamos usar o withColumn, para criar uma coluna com caracteres reservados.

In [31]:
# Não precisamos de caracteres de escape aqui porque o primeiro argumento para withColumn é apenas uma string 
# para o novo nome da coluna.
dfWithLongColName = df.withColumn("This Long Column-Name", expr("ORIGIN_COUNTRY_NAME"))

In [33]:
# Neste exemplo, no entanto, precisamos usar backticks porque estamos fazendo 
# referência a uma coluna em uma expressão.
dfWithLongColName.selectExpr("`This Long Column-Name`", "`This Long Column-Name` as `new col`").show(5)

+---------------------+-------------+
|This Long Column-Name|      new col|
+---------------------+-------------+
|              Romania|      Romania|
|              Croatia|      Croatia|
|              Ireland|      Ireland|
|        United States|United States|
|                India|        India|
+---------------------+-------------+
only showing top 5 rows



Podemos nos referir a colunas com caracteres reservados (e não ignorá-los) se estivermos fazendo uma referência explícita de string para coluna, que é interpretada como literal em vez de expressão. Precisamos apenas "escapar" de expressões que usam caracteres reservados ou palavras-chave

In [35]:
dfWithLongColName.select(expr("`This Long Column-Name`")).columns

['This Long Column-Name']

Material baseado em exemplos do livro __Spark - The Definitive Guide. Bill Chambers e Matei Zaharia__